In [ ]:
import findspark
findspark.init("/usr/hdp/current/spark2-client/")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f

In [ ]:
spark = SparkSession.builder \
.master("yarn") \
.appName("ReadFromConsole") \
.getOrCreate()

In [ ]:
lines = spark.readStream \
.format("socket") \
.option("host","sandbox-hdp.hortonworks.com") \
.option("port", 9999) \
.load()

In [ ]:
words = lines.select(f.explode(f.split(f.col("value"), "\s+")).alias("word"))

In [ ]:
word_counts = words.groupBy("word").count().sort(f.desc("count"))

In [1]:
# Sandbox komut satırından nc -l 9999 ile netcat server başlatılır

In [ ]:
query = word_counts.writeStream \
.outputMode("complete") \
.format("console") \
.start()

In [ ]:
query.awaitTermination()

In [ ]:
# nc consoldan girilen girdilerin streaming ile işlenmiş hali jupyter consoldan gözlenir

In [ ]:
query.stop()